In [16]:
import json
import math

In [124]:
def load_json(file):
    f = open(file, 'r')
    return json.load(f)

In [3]:
def write_json(data):
    encoded_data = json.JSONEncoder(indent=2).encode(data)
    
    f = open('mud_of_babel_data.json', 'w')
    f.write(encoded_data)

In [4]:
def ordinal(integer):
    int_to_string = str(integer)
    if int_to_string == '1' or int_to_string == '-1':
        return int_to_string+'st'
    elif int_to_string == '2' or int_to_string == '-2':
        return int_to_string+'nd'
    elif int_to_string == '3' or int_to_string == '-3':
        return int_to_string+'rd'
    elif int_to_string[-1] == '1' and int_to_string[-2] != '1':
        return int_to_string+'st'
    elif int_to_string[-1] == '2' and int_to_string[-2] != '1':
        return int_to_string+'nd'
    elif int_to_string[-1] == '3' and int_to_string[-2] != '1':
        return int_to_string+'rd'
    else:
        return int_to_string+'th'

In [45]:
def doors_dict():
    return {
        "S": "South",
        "N": "North",
        "E": "East",
        "W": "West",
        "SE": "SouthEast",
        "NW": "NorthWest",
        "SW": "SouthWest",
        "NE": "NorthEast"
    }

In [5]:
def build_basic_game_info():
    return {
        'commandCounter' : 0,
        'gameOver' : False,
        'introText' : '\n\nWelcome to the MUD of Babel!\n',
        'outroText' : 'Thanks For playing!',
        'player' : {
            'currentLocation' : '',
            'inventory' : {},
            'lightSource' : False
        },
        'map': {}
    }

In [6]:
def build_room(key, name, description, exits={}, interactables={}, items={}):
    return {
        key: {
            'firstVisit' : True,
            'displayName' : name,
            'description' : description,
#             'setup' : 'function(){end();}',
            'interactables' : interactables,
            'items' : items,
            'exits' : exits
        }
    }


In [175]:
def build_floor(subject, code, floor, children, shape, is_top=False):
    '''
    The N-th floor of a building with up to eight doorways
    Doorways are organized in a S/N/E/W/SE/NW/SW/NE order
    Each floor is shaped depending on the shape of the building
    '''
    _child_count = len(children)
    doors_list = doors_dict()
    floor_name = "“%s Building”, %s Floor" % (subject, ordinal(floor))
    floor_code = code
    
    exits = {}
    rooms = {}

    be_str = 'are' if _child_count != 1 else 'is'
    room_str = 'rooms' if _child_count != 1 else 'room'
    description = 'You are in the %s floor. There %s %s %s:\n' % (ordinal(floor), be_str, _child_count, room_str)
    entrances = []
    for index, item in enumerate(children):
        key, val = item
        exit_geo =  list(doors_dict().values())[index]
        exit_key =  "%s-%s" % (code, val['id'])
        entrances.append('[%s] “%s Room”' % (exit_geo.ljust(7).rjust(9), val['subject']))
        exits.update({
            exit_key : {
                'displayName' : exit_geo,
                'destination' : exit_key
            }
        })

        room_subject = val['subject']
        room_exit = {
            'exit' : {
                'displayName' : 'Exit',
                'destination' : floor_code
            }
        }
        room_description = "Description for “%s Room”. You can exit to the floor foyer." % room_subject
        room_description = "%s\n[%s] Floor foyer" % (room_description, 'Exit'.ljust(7).rjust(9))
        room = build_room(key=exit_key, name=room_subject, description=room_description, exits=room_exit)
        rooms.update(room)

    if floor > 1:
        floor_code = '%s_%s' % (floor, code)
        down_code = ("%s_%s" % (floor-1, code)) if floor-1 > 1 else code
        exits.update({
            'down' : {
                'displayName' : 'Down',
                'destination' : down_code
            }
        })
        entrances.append('[%s] %s floor' % ('Down'.ljust(7).rjust(9), ordinal(floor-1)))

    if (is_top is not False):
        # still floors left
        up_code = "%s_%s" % ((floor+1), code)
        exits.update({
            'up' : {
                'displayName' : 'Up',
                'destination' : up_code
            }
        })
        entrances.append('[%s] %s floor' % ('Up'.ljust(7).rjust(9), ordinal(floor+1)))

    description = '%s\n%s' % (description, '\n'.join(entrances))
    rooms.update(build_room(key=floor_code, name=floor_name, description=description, exits=exits))
    return rooms

In [176]:
def build_building(plaza_code, subject, code, children, plaza_subject):
    '''
    Each building is shaped depending on the amount of child classes it has:
    - five or more: octagonal shape with multiple floors (if nine or more children)
    - one to four: square
    '''
    _child_count = len(children)
    shape = "square" if _child_count < 5 else "octagonal"
    cardinals = ', '.join(list(doors_dict().values())[0:4]) if _child_count < 5 else ', '.join(doors_dict().values())
    floors = 1

    rooms = {}
    entrances = []
    exits = {
        code : {
            'displayName' : 'Exit',
            'destination' : plaza_code
        }
    }
    
    if _child_count > 8:
        floors = math.ceil(_child_count / 8)
        exits.update({
            'up' : {
                'displayName' : 'Up',
                'destination' : '%s_%s' % (2, code)
            }
        })

    '''
    for floors 1-n:
    loop thru children in steps of 8
    '''
    for i in range(0, len(children), 8):
        # get the set of 8 (or fewer)
        floor_children = list(children)[i:(i+8)]
        # check if is last floor
        is_top = (i + 8 < len(children))
        # build the floor
        rooms.update(build_floor(children=floor_children, shape=shape, code=code, floor=int(i/8)+1, subject=subject, is_top=is_top))

    # loop for the first floor (to also use for exit descriptions)
    for index, item in enumerate(list(children)[0:8]):
        key, val = item
        exit_geo =  list(doors_dict().values())[index]
        exit_key = "%s-%s" % (code, val['id'])
        entrances.append('[%s] “%s Room”' % (exit_geo.ljust(7).rjust(9), val['subject']))
        exits.update({
            exit_key : {
                'displayName' : exit_geo,
                'destination' : exit_key
            }
        })

    if _child_count > 8:
        entrances.append('[%s] %s floor' % ('Up'.ljust(7).rjust(9), ordinal(2)))

    entrances.append('[%s] %s Plaza' % ('Exit'.ljust(7).rjust(9), plaza_subject))

    be_str = 'are' if _child_count != 1 else 'is'
    room_str = 'rooms' if _child_count != 1 else 'room'
    floor_str = 'floors' if floors != 1 else 'floor'
    first_str = ' You are in the first floor.' if floors != 1 else ''
    building_name = "“%s Building” Main Lobby" % subject
    building_floor = ' in %s %s' % (floors, floor_str) if floors > 1 else ''
    description = 'There %s %s %s throughout this building%s.%s The building is %s-shaped with walls facing %s. The %s in this floor %s:\n' % (be_str, _child_count, room_str, building_floor, first_str, shape, cardinals, room_str, be_str)
    description = '%s\n%s' % (description, '\n'.join(entrances))

    # build room for main lobby
    rooms.update(build_room(key=code, name=building_name, description=description, exits=exits))
    return rooms

In [187]:
def build_plaza(subject, code, children):
    '''
    Plazas are surrounded by buildings representing all the child classes.
    The amount of buildings determines the decorations in the plaza and the diameter.
    '''
    _child_count = len(children)
    plaza_code = 'Plaza-%s' % code
    name = '%s Plaza' % subject
    be_str = 'are' if _child_count != 1 else 'is'
    building_str = 'buildings' if _child_count != 1 else 'building'
    their_str = 'their' if _child_count != 1 else 'its'
    description = 'You are in an plaza surrounded by %s %s conforming %s.\nThe %s %s accessible via %s classification code, visible above the main door:\n' % (_child_count, building_str, subject, building_str, be_str, their_str)
    exits = {}
    entrances = []
    for key, val in children:
        exit_key = "%s-%s" % (code, val['id'])
        stories = '%s %s' % (math.ceil(len(val['children']) / 8), 'story' if len(val['children']) <= 8 else 'stories')
        entrances.append('[%s] (%s) “%s Building”' % (exit_key.ljust(6).rjust(7), stories, val['subject']))
        exits.update({
            exit_key : {
                'displayName' : exit_key,
                'destination' : exit_key
            }
        })

    entrances.append('[%s] Main Library Grounds' % ('Main'.ljust(6).rjust(7)))
    exits.update({
        'main' : {
            'displayName' : 'Main',
            'destination' : 'MAIN'
        }
    })
    description = '%s\n%s' % (description, '\n'.join(entrances))
    return build_room(key=plaza_code, name=name, description=description, exits=exits)

In [184]:
def build_plazas(class_json):
    class_code = class_json['code']
    subject = class_json['subject']
    
    plaza_code = 'Plaza-%s' % class_code

    rooms = build_plaza(subject=subject, code=class_code, children=class_json['children'].items())

    for child_key, section in class_json['children'].items():
        section_code = '%s-%s' % (class_code, child_key)
        rooms.update(build_building(plaza_code=plaza_code, plaza_subject=subject, subject=section['subject'], code=section_code, children=section['children'].items()))
    
    return rooms

In [185]:
def build_main_square(plazas):
    '''
    Main game square. All librarians start here.
    '''
    _child_count = len(plazas)
    code = 'MAIN'
    name = 'Main Library Grounds'
    description = 'You are in an expansive area where %s smaller roads meet. Each road has a small sign next to it, indicating where it leads:\n' % (_child_count)
    exits = {}
    entrances = []
    for item in plazas:
        exit_geo = "Plaza %s" % (item['code'])
        exit_key = "Plaza-%s" % (item['code'])
        entrances.append('[%s] “%s Plaza” (%s)' % (exit_geo, item['subject'], "%sm" % (item['child_count'] * 50) if (item['child_count'] * 50) < 1000 else "%skm" % (item['child_count'] * 50 / 1000)))
        exits.update({
            exit_key : {
                'displayName' : exit_geo,
                'destination' : exit_key
            }
        })
    description = '%s\n%s' % (description, '\n'.join(entrances))
    return build_room(key=code, name=name, description=description, exits=exits)

In [188]:
def parse_lcc_file(filename):
    game_info = build_basic_game_info()

    raw_json = load_json(filename)
    
    rooms = {}
    plazas = []
    
    for key, val in raw_json.items():
        plazas.append({
            'code': val['code'],
            'subject': val['subject'],
            'child_count': len(val['children'])
        })
        rooms.update(build_plazas(val))
    
    rooms.update(build_main_square(plazas))
    first_room = 'MAIN'
    game_info['map'] = rooms
    game_info['player']['currentLocation'] = first_room
    write_json(game_info)

parse_lcc_file('lcc_simple_count.json')

In [140]:
import sys; from PIL import Image; import numpy as np

chars = np.asarray(list(" ░▒▓█"))
chars = np.asarray(list(" .;-:!>7?CO$QHNM"))
# chars = np.asarray(list(" .:-=+*#%@"))
# chars = np.asarray(list(" .'`^\",:;Il!i><~+_-?][}{1)(|/tfjrxnuvczXYUJCLQ0OZmwqpdbkhao*#MW&8%B@$"))

# if len(sys.argv) != 4: print( 'Usage: ./asciinator.py image scale factor' ); sys.exit()
f, SC, GCF, WCF = "library.jpg", float(0.1), float(1.5), 6/4

img = Image.open(f).convert('LA')
S = ( round(img.size[0]*SC*WCF), round(img.size[1]*SC) )
img = np.sum( np.asarray( img.resize(S) ), axis=2)
img -= img.min()
img = (1.0 - img/img.max())**GCF*(chars.size-1)

print( "\n".join( ("".join(r) for r in chars[img.astype(int)]) ) )

?>>!:-;->7?OOCCC??>CC>>!::??>::-::->:::---;;:;:-:--:::!!!!::-!->??!:::!>OQ7!?O$H?7!>: ..;;->!!>>
C?>!::;.!!!>?OCQ$>>:OO7C$77!:::::>--;:CO?C??C?C7777?7O7!!;:7-!:->77$7!C$$:7OQQHO?>!!   ;;-!>:>!!
OC?>>:-;;!!!!!C$OOCC!$H??>::::7!:-:!777?>?7!!7??>>??777?C:::!7!:---?7?OO!O>CQ?O!!:::...;-;:>:-:-
$$?7!!-;.>!!:!:7C7$C77$O!!:>7:::!7?7?!:-?7>>77?>>!!!>:>?77C::!:!>--:CCC!>Q$$?C!:::: ...;:::>QHQ>
QCC77>:;;:!!:-!!:CO?OO?O!?!:--!777>::7-!!::::!!:!:!!>!:!:;!>C!:!-!7C:CC?7$??!!:::--. --::-!?Q$O:
QO$>77>:;;:::!!-:.:?7$C?!O?77C>!::;-::>C?7C??CCC?7C??O>;->-:!??>>!?QCO$H??:::::----..;;;;>-Q$C$N
QQN7>>!:-;:::!::;.-:!$OO>>O?C!:-7::7??C!>?7????CC??C>77??::!>;-??HO!7??C!-  ;:-;;-..;;;;:>>$O7$?
!?>$77>!--::-:!;;. ::!C>$?7CO!--:!O?-:>-!C????$77?7C!-7!>??>-:?!O?>$O>7!;-  .:-;;;;.;;;;->$HOH$H
>::>?>>:-----::!-..--::!CCOO:$7C?!:7:-7?CC????O??77$OC7:!!:???>COOO$C!!-; .-::;;;.;..;;-?:$OHQNC
NNO!>!!:--;;-;7!:-.;;-:.-?>$::$7::-:?C>>!COO$OC$$O$$>?7CO:-?-7Q!:7!7;-;;..;-!:;;..;.;;.->NQQOMNQ
MHONQ!::--..;-?!!:;.-;-;.::??O